Look for shapes that have the same color as their neighbors, and change them to a new random color until there are no collisions. 

In [1]:
import random
import pandas as pd
import geopandas as gp

In [2]:
df = gp.read_file('smd.geojson')

df.rename(columns={
    'Color_Category': 'map_color_id'
    , 'SMD_ID': 'smd_id'
}, inplace=True)

df.sort_values(by='smd_id', inplace=True)

df['neighbors'] = None
df['num_neighbors'] = None
df['neighbor_colors'] = None
df['has_collision'] = False

possible_colors = list(range(1,13))

In [3]:
for idx, row in df.iterrows():

    # get 'not disjoint' countries
    neighbors = df[~df.geometry.disjoint(row.geometry)].smd_id.tolist()

    # remove own name from the list
    neighbors = [ name for name in neighbors if row.smd_id != name ]

    # add names of neighbors as NEIGHBORS value
    df.loc[idx, 'neighbors'] = ", ".join(neighbors)
    df.loc[idx, 'num_neighbors'] = len(neighbors)

In [4]:
# df.groupby('num_neighbors').size()

In [5]:
def assess_collisions(df):
    """Mark True for districts with collisions"""
    
    for idx, row in df.iterrows():
        neighbors = row['neighbors'].split(', ')
        neighbor_colors = [df.loc[df.smd_id == n, 'map_color_id'].values[0] for n in neighbors]

        df.loc[idx, 'neighbor_colors'] = ", ".join([str(n) for n in neighbor_colors])

        if row['map_color_id'] in neighbor_colors:
            df.loc[idx, 'has_collision'] = True
    
    num_collisions = df['has_collision'].sum()
    print(f'Current collisions: {num_collisions}')

    return df, num_collisions

In [6]:
def change_one_district_color(df):
    """Change color for one district to an available color"""
    
    smd_to_change = df[df['has_collision']].head(1)['smd_id'].values[0]

    row = df[df['smd_id'] == smd_to_change]
    
    old_color = row['map_color_id'].values[0]

    neighbor_colors_str = row['neighbor_colors'].values[0].split(', ')
    neighbor_colors = [int(n) for n in neighbor_colors_str]
    
    available_colors = [c for c in possible_colors if c not in neighbor_colors]
    new_color = random.choice(available_colors)

    df.loc[row.index, 'map_color_id'] = new_color
    df.loc[row.index, 'has_collision'] = False
    
    print(f'District {smd_to_change} changed from {old_color} to color {new_color}')
    
    return df

In [7]:
df, num_collisions = assess_collisions(df)

num_iterations = 100
i = 0

while num_collisions != 0 and i < num_iterations:
    i += 1
    print()
    
    df = change_one_district_color(df)
    df, num_collisions = assess_collisions(df)

Current collisions: 64

District 1A10 changed from 10 to color 3
Current collisions: 63

District 1B05 changed from 5 to color 9
Current collisions: 62

District 1B10 changed from 10 to color 8
Current collisions: 61

District 2A01 changed from 2 to color 1
Current collisions: 60

District 2A02 changed from 3 to color 1
Current collisions: 59

District 2A03 changed from 4 to color 3
Current collisions: 58

District 2B02 changed from 11 to color 6
Current collisions: 57

District 2B05 changed from 2 to color 2
Current collisions: 56

District 2B06 changed from 3 to color 10
Current collisions: 55

District 2B08 changed from 5 to color 5
Current collisions: 54

District 2C01 changed from 7 to color 4
Current collisions: 53

District 2C02 changed from 8 to color 2
Current collisions: 52

District 2C03 changed from 9 to color 8
Current collisions: 51

District 2D02 changed from 11 to color 11
Current collisions: 50

District 2E05 changed from 4 to color 6
Current collisions: 49

District 3

In [13]:
df[['smd_id', 'map_color_id', 'geometry', 'neighbors']].to_file('smd.geojson', driver='GeoJSON')

In [15]:
df[['smd_id', 'map_color_id', 'neighbors']].to_csv('smd.csv', index=False)

In [9]:
# df[df['smd_id'] == '1A10']

In [19]:
df[(df['smd_id'] == '2C02') | (df['smd_id'] == '6E05') ]

,OBJECTID,smd_id,ANC_ID,Shape_Leng,Shape_Area,map_color_id,geometry,neighbors,num_neighbors,neighbor_colors,has_collision
195,196,2C02,2C,1317.401612,91335.638459,2,"MULTIPOLYGON (((-77.01408 38.89944, -77.01408 ...","2C01, 2C03, 6C02, 6E05, 6E07",5,"4, 8, 4, 3, 10",False
231,232,6E05,6E,1698.773461,150003.694796,3,"MULTIPOLYGON (((-77.01620 38.90021, -77.01620 ...","2C01, 2C02, 2F06, 6E04, 6E07",5,"4, 2, 1, 2, 10",False
